In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [3]:
import os
import pandas as pd
import shutil

# Paths
base_dir = './detection_tunisian_food'
csv_file = os.path.join(base_dir, '_annotations.csv')
image_dir = os.path.join(base_dir, 'valid')  # Assuming images are in 'test'

# Load the CSV file
df = pd.read_csv('./detection_tunisian_food/valid/_annotations.csv')


# Create directories for each class
class_names = df['class'].unique()
for class_name in class_names:
    class_dir = os.path.join(base_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

# Move images to respective class folders
for _, row in df.iterrows():
    filename = row['filename']
    class_name = row['class']
    src_path = os.path.join(image_dir, filename)
    dest_path = os.path.join(base_dir, class_name, filename)
    
    # Check if the file exists before moving
    if os.path.exists(src_path):
        shutil.move(src_path, dest_path)
    else:
        print(f"File not found: {src_path}")

print("Images have been organized into class folders.")


File not found: ./detection_tunisian_food\valid\recipe_image_9_12_jpg.rf.00629e519f18860ec5413901ae60ef0e.jpg
File not found: ./detection_tunisian_food\valid\images-5-_jpeg.rf.05d093ba9376b96ca452f3b165876f0a.jpg
File not found: ./detection_tunisian_food\valid\recipe_image_14_15_jpg.rf.30189aea4fad1c2b80c3438b73fbffd4.jpg
File not found: ./detection_tunisian_food\valid\Baklawa-facile_jpg.rf.3d974c3e09ab69cc774a0c7534a8e78b.jpg
File not found: ./detection_tunisian_food\valid\recipe_image_9_96_jpg.rf.8e24cdbde809dee8e5c6451b5dd247cb.jpg
File not found: ./detection_tunisian_food\valid\recipe_image_9_96_jpg.rf.8e24cdbde809dee8e5c6451b5dd247cb.jpg
File not found: ./detection_tunisian_food\valid\6_jpeg.rf.9249ca9bba69f9228529d829097259ab.jpg
File not found: ./detection_tunisian_food\valid\frecasee_jpeg.rf.9dba78161833e1474410e00e7a370134.jpg
File not found: ./detection_tunisian_food\valid\498382_jpg.rf.a5638d372dc69d7f3a9ba3c778becce2.jpg
File not found: ./detection_tunisian_food\valid\49838

In [5]:
# Define directories
train_dir = './detection_tunisian_food/train'
valid_dir = './detection_tunisian_food/valid'
test_dir = './detection_tunisian_food/test'

# Image dimensions
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


NameError: name 'ImageDataGenerator' is not defined

In [5]:
# Définir les chemins
base_dir = './detection_tunisian_food'
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')
test_dir = os.path.join(base_dir, 'test')

# Générateurs de données avec augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

# Créer les générateurs
train_data = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(224, 224), 
    batch_size=32, 
    class_mode='categorical'
)

valid_data = valid_datagen.flow_from_directory(
    valid_dir, 
    target_size=(224, 224), 
    batch_size=32, 
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(224, 224), 
    batch_size=32, 
    class_mode='categorical'
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [ ]:
# Charger le modèle MobileNetV2
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3), 
    include_top=False, 
    weights='imagenet'
)

# Geler les couches convolutives
base_model.trainable = False

# Ajouter une tête de classification personnalisée
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Afficher le résumé
model.summary()


In [ ]:
# Entraîner uniquement les couches ajoutées
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10,
    steps_per_epoch=len(train_data),
    validation_steps=len(valid_data),
)


In [ ]:
# Débloquer les couches supérieures de MobileNetV2
base_model.trainable = True

# Régler un taux d'apprentissage réduit
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

# Fine-tuning
fine_tune_history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10,
    steps_per_epoch=len(train_data),
    validation_steps=len(valid_data),
)


In [ ]:
# Évaluer sur le set de test
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Sauvegarder le modèle
model.save('tunisian_food_mobilenetv2.h5')


In [ ]:
# Courbes de précision
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

# Courbes de perte
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()
